In [ ]:
!pip install torch transformers datasets sentencepiece
!pip install tensorboard scikit-learn psutil sacrebleu rouge-score tensorflow_datasets pytorch-lightning matplotlib git-python faiss-cpu streamlit elasticsearch nltk pandas datasets fire pytest conllu sentencepiece protobuf
!pip install jax jaxlib
!#pip install torch-lr-finder
!pip install wandb
!pip install progressbar

In [ ]:
# imports 
import logging
from transformers import PegasusTokenizerFast, PegasusForConditionalGeneration,PegasusConfig,AutoTokenizer,AutoModelForSeq2SeqLM
import datasets

import torch
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F

from typing import Callable, Dict, Iterable, List, Tuple, Union
from transformers import EvalPrediction, PreTrainedTokenizer
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

from transformers import AdamW
import wandb
import gc
import os
import numpy as np
import random

import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
#seeding
def seed_torch(seed=1029):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(2021)

In [ ]:
# Logging

logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

wandb.init()

device = 'cuda' if torch.cuda.is_available() else 'cpu'


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from contextlib import contextmanager
import time
import string
import warnings
warnings.filterwarnings('ignore')

@contextmanager
def timer(msg):
    t0 = time.time()
    print(f'[{msg}] start.')
    yield
    elapsed_time = time.time() - t0
    print(f'[{msg}] done in {elapsed_time / 60:.2f} min.')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Teacher model and Tokenizer

tokenizer = AutoTokenizer.from_pretrained('google/pegasus-gigaword')

#teacher = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-gigaword')
#copy_teacher = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-large')

05/11/2021 13:51:10 - INFO - filelock -   Lock 140516773774160 acquired on /root/.cache/huggingface/transformers/5208455d9c895ad544faafb54e3504b267d9ddb014088d7e96a779680693bbe9.0bf641a51074a535c2c9b0ff302b36ccda971f61f75ec45c69172780fbb4acd8.lock


05/11/2021 13:51:10 - INFO - filelock -   Lock 140516773774160 released on /root/.cache/huggingface/transformers/5208455d9c895ad544faafb54e3504b267d9ddb014088d7e96a779680693bbe9.0bf641a51074a535c2c9b0ff302b36ccda971f61f75ec45c69172780fbb4acd8.lock


05/11/2021 13:51:10 - INFO - filelock -   Lock 140516786879120 acquired on /root/.cache/huggingface/transformers/78453cb54b63e849802b861b9956256d8d7cf066a5c96d3a1738aed311ca94b1.1acf68c74589da6c7fa3548093824dfc450a54637f4356929bbfea7e294a68f8.lock


05/11/2021 13:51:11 - INFO - filelock -   Lock 140516786879120 released on /root/.cache/huggingface/transformers/78453cb54b63e849802b861b9956256d8d7cf066a5c96d3a1738aed311ca94b1.1acf68c74589da6c7fa3548093824dfc450a54637f4356929bbfea7e294a68f8.lock


05/11/2021 13:51:12 - INFO - filelock -   Lock 140519492643792 acquired on /root/.cache/huggingface/transformers/e51f586463feaa77af7ae0fcef66e04a5eea479b942e194dc4b1257c05f7dfab.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82.lock


05/11/2021 13:51:12 - INFO - filelock -   Lock 140519492643792 released on /root/.cache/huggingface/transformers/e51f586463feaa77af7ae0fcef66e04a5eea479b942e194dc4b1257c05f7dfab.294ebaa4cd17bb284635004c92d2c4d522ec488c828dcce0c2471b6f28e3fe82.lock


05/11/2021 13:51:12 - INFO - filelock -   Lock 140516792660560 acquired on /root/.cache/huggingface/transformers/cf79f466aa9ca1cd51b5cbb68aeb80dd1e9f91e4ed701c6ae926462b0fc83a90.a631152251c90783adb88a666e938f738f41e28867e9320503e97539046612cf.lock


05/11/2021 13:51:13 - INFO - filelock -   Lock 140516792660560 released on /root/.cache/huggingface/transformers/cf79f466aa9ca1cd51b5cbb68aeb80dd1e9f91e4ed701c6ae926462b0fc83a90.a631152251c90783adb88a666e938f738f41e28867e9320503e97539046612cf.lock


In [ ]:
#Student configuration

import warnings
import torch
from torch import nn
from typing import Optional, Tuple, List, Union
from transformers import PegasusModel, PegasusConfig, PegasusForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedModel
from transformers import SummarizationPipeline

students_config_book = {
    '2': PegasusConfig(encoder_layers=2, decoder_layers=2),
    '4': PegasusConfig(encoder_layers=4, decoder_layers=4),
    '6': PegasusConfig(encoder_layers=6, decoder_layers=6),
    '8': PegasusConfig(encoder_layers=8, decoder_layers=8),
    '10': PegasusConfig(encoder_layers=10, decoder_layers=10),
    '12': PegasusConfig(encoder_layers=12, decoder_layers=12),
    '16': PegasusConfig(encoder_layers=16, decoder_layers=16)
}


LAYERS_TO_COPY = {   
    4:{
        1: [0],
        2: [0, 3],
        3: [0, 1, 3],
        4: [0, 1, 2, 3],
    },
    8:{
        1: [0],
        2: [0, 7],
        3: [0, 4, 7],
        4: [0, 3, 6, 7],
        6: [0, 2, 3, 5, 6, 7],
        8: list(range(8)),  
    },    
    12: {
        1: [0],
        2: [0, 11],
        3: [0, 6, 11],
        4: [0, 4, 9, 11],
        6: [0, 2, 5, 8, 10, 11],
        8: [0, 1, 3, 5, 7, 9, 10, 11],
        12: list(range(12)),  
    },
    16: {  # maps  num layers in student -> which teacher layers to copy
        1: [0],
        2: [0, 15],
        3: [0, 8, 15],
        4: [0, 5, 10, 15],
        6: [0, 3, 6, 9, 12, 15],
        8: [0, 2, 4, 6, 8, 10, 12, 15],
        9: [0, 1, 3, 5, 7, 9, 11, 13, 15],
        12: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15],
        16: list(range(16)),
    },}
LAYERS_TO_SUPERVISE = {
    # maps  num layers in student -> which teacher layers to copy.
    8: {1: [5], 2: [3, 5], 3: [1, 4, 5], 4: [1, 2, 4, 5]},
    12: {1: [11], 2: [5, 11], 3: [3, 7, 11], 4:[1, 3, 7, 11],6: [1, 3, 5, 8, 10, 11], 8:[1,2,3,5,7,8,9,11] },
    16: {1: [15], 4: [4, 9, 12, 15], 8: [1, 3, 5, 7, 9, 11, 13, 15], 12:[1,2,3,5,7,8,9,11,12,13,14,15]},
}


def copy_layers(src_layers: nn.ModuleList, dest_layers: nn.ModuleList, layers_to_copy) -> None:
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(
        layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

# Copied from transformers.models.bart.modeling_bart.shift_tokens_right


def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    assert pad_token_id is not None, "self.model.config.pad_token_id has to be defined."
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids


def pick_layers_to_copy(n_student, n_teacher):
    try:
        val = LAYERS_TO_COPY[n_teacher][n_student]
        return val
    except KeyError:
        if n_student != n_teacher:
            warnings.warn(
                f"no hardcoded layers to copy for teacher {n_teacher} -> student {n_student}, defaulting to first {n_student}"
            )
        return list(range(n_student))

def get_layers_to_supervise(n_student, n_teacher) -> List[int]:
    """Used or the --supervise_forward kwarg"""
    if n_student > n_teacher:
        raise ValueError(f"Cannot perform intermediate supervision for student {n_student} > teacher {n_teacher}")
    elif n_teacher == n_student:
        return list(range(n_teacher))
    elif n_student == 1:
        return [n_teacher - 1]
    else:
        return LAYERS_TO_SUPERVISE[n_teacher][n_student]

def create_student_with_configuration(teacher,
                                      e=None,
                                      d=None,
                                      copy_first_teacher_layers = False,
                                      save_path='./student'):

    teacher.eval()
    teacher_e, teacher_d = teacher.config.encoder_layers, teacher.config.decoder_layers
    init_kwargs = teacher.config.to_diff_dict()
    if e is None:
        e = teacher_e
    if d is None:
        d = teacher_d
    init_kwargs.update({"encoder_layers": e, "decoder_layers": d})
    student_cfg = teacher.config_class(**init_kwargs)
    student = AutoModelForSeq2SeqLM.from_config(student_cfg)
    # Start by copying the full teacher state dict this will copy the first N teacher layers to the student.
    info = student.load_state_dict(teacher.state_dict(), strict=False)
    # every student key should have a teacher keys.
    assert info.missing_keys == [], info.missing_keys

    if copy_first_teacher_layers:  # Our copying is done. We just log and save
        e_layers_to_copy, d_layers_to_copy = list(range(e)), list(range(d))
        #student.save_pretrained(save_path)
        return student, e_layers_to_copy, d_layers_to_copy

    # Decide which layers of the teacher to copy. Not exactly alternating -- we try to keep first and last layer.
    e_layers_to_copy: List[int] = pick_layers_to_copy(e, teacher_e)
    d_layers_to_copy: List[int] = pick_layers_to_copy(d, teacher_d)

    copy_layers(teacher.model.encoder.layers,
                student.model.encoder.layers, e_layers_to_copy)
    copy_layers(teacher.model.decoder.layers,
                student.model.decoder.layers, d_layers_to_copy)

    student.config.init_metadata = dict(
        teacher_type=teacher.config.model_type,
        copied_encoder_layers=e_layers_to_copy,
        copied_decoder_layers=d_layers_to_copy,
    )
    #student.save_pretrained(save_path)
    # Save information about copying for easier reproducibility

    return student, e_layers_to_copy, d_layers_to_copy
#student = create_student_with_configuration(teacher,
#                                      e=4,
#                                      d=4,
#                                      copy_first_teacher_layers = False,
#                                      save_path='./student')
#import gc
#del copy_teacher
gc.collect()


8

# Data Loading

In [ ]:
#student

In [ ]:
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels["input_ids"])


def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):
  """
  Prepare input data for model fine-tuning
  """
  tokenizer = AutoTokenizer.from_pretrained(model_name)

  prepare_val = False if val_texts is None or val_labels is None else True
  prepare_test = False if test_texts is None or test_labels is None else True

  def tokenize_data(texts, labels):
    encodings = tokenizer(texts, truncation=True, padding='longest')
    decodings = tokenizer(labels, truncation=True, padding='longest')
    dataset_tokenized = PegasusDataset(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  val_dataset = tokenize_data(val_texts, val_labels) if prepare_val else None
  test_dataset = tokenize_data(test_texts, test_labels) if prepare_test else None

  return train_dataset, val_dataset, test_dataset

In [ ]:
def get_dataloaders(train_texts, train_labels,valid_texts = None, valid_labels = None ,test_texts = None, test_labels = None, bsz = 24):
    train_dataset, test_dataset, valid_dataset = prepare_data('google/pegasus-gigaword', train_texts, train_labels,valid_texts, valid_labels,test_texts, test_labels)
    train_dataloader = DataLoader(train_dataset,batch_size=bsz,num_workers = 2)
    test_dataloader = DataLoader(test_dataset,batch_size=bsz,num_workers = 2)
    validation_dataloader = DataLoader(valid_dataset,batch_size=bsz,num_workers = 2)
    return train_dataloader, test_dataloader, validation_dataloader

In [ ]:
with timer('load data'):

  dataset = datasets.load_dataset('gigaword')

  #source data
  train_texts, train_labels = dataset['train']['document'][:250000], dataset['train']['summary'][:250000]
  valid_texts, valid_labels = dataset['validation']['document'][:10000], dataset['validation']['summary'][:10000]
  test_texts, test_labels = dataset['test']['document'], dataset['test']['summary']


  del dataset 
  gc.collect()

  bsz = 24
  train_dataloader , test_dataloader, validation_dataloader = get_dataloaders(train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels,bsz)

  gc.collect()


[load data] start.


05/11/2021 13:51:15 - WARNING - datasets.builder -   Using custom data configuration default


Dataset gigaword downloaded and prepared to /root/.cache/huggingface/datasets/gigaword/default/1.2.0/ea83a8b819190acac5f2dae011fad51dccf269a0604ec5dd24795b64efb424b6. Subsequent calls will reuse this data.
[load data] done in 3.23 min.


In [ ]:
next(iter(train_dataloader))

{'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'input_ids': tensor([[14937,  1034,   116,  ...,     0,     0,     0],
         [  134,   583,   228,  ...,     0,     0,     0],
         [73013,  2853,  2127,  ...,     0,     0,     0],
         ...,
         [  110, 57975, 22109,  ...,     0,     0,     0],
         [  228, 44909,   195,  ...,     0,     0,     0],
         [ 2363, 72265,  2853,  ...,     0,     0,     0]]),
 'labels': tensor([[73013,   582,   728,  ...,     0,     0,     0],
         [  134,   583,   228,  ...,     0,     0,     0],
         [73013,  5446,   686,  ...,     0,     0,     0],
         ...,
         [  110, 57975, 22109,  ...,     0,     0,     0],
         [  228,  1564, 13006,  ...,     0,     0,     0],
         [  110, 28546,   216,  ...,     0,     0,    

# Utils

In [ ]:
!pip install nltk
import re

from filelock import FileLock


try:
    import nltk

    NLTK_AVAILABLE = True
except (ImportError, ModuleNotFoundError):
    NLTK_AVAILABLE = False

if NLTK_AVAILABLE:
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)


def add_newline_to_end_of_each_sentence(x: str) -> str:
    """This was added to get rougeLsum scores matching published rougeL scores for BART and PEGASUS."""
    re.sub("<n>", "", x)  # remove pegasus newline char
    assert NLTK_AVAILABLE, "nltk must be installed to separate newlines between sentences. (pip install nltk)"
    return "\n".join(nltk.sent_tokenize(x))

05/11/2021 13:54:32 - INFO - filelock -   Lock 140519409410448 acquired on .lock
05/11/2021 13:54:33 - INFO - filelock -   Lock 140519409410448 released on .lock


In [ ]:

## ROUGE Utils
from rouge_score import rouge_scorer, scoring

ROUGE_KEYS = ["rouge1", "rouge2", "rougeL", "rougeLsum"]


def extract_rouge_mid_statistics(dct):
    new_dict = {}
    for k1, v1 in dct.items():
        mid = v1.mid
        new_dict[k1] = {stat: round(getattr(mid, stat), 4) for stat in ["precision", "recall", "fmeasure"]}
    return new_dict


def calculate_rouge(
    pred_lns: List[str],
    tgt_lns: List[str],
    use_stemmer=True,
    rouge_keys=ROUGE_KEYS,
    return_precision_and_recall=False,
    bootstrap_aggregation=True,
    newline_sep=True,
) -> Dict:
    """Calculate rouge using rouge_scorer package.

    Args:
        pred_lns: list of summaries generated by model
        tgt_lns: list of groundtruth summaries (e.g. contents of val.target)
        use_stemmer:  Bool indicating whether Porter stemmer should be used to
        strip word suffixes to improve matching.
        rouge_keys:  which metrics to compute, defaults to rouge1, rouge2, rougeL, rougeLsum
        return_precision_and_recall: (False) whether to also return precision and recall.
        bootstrap_aggregation: whether to do the typical bootstrap resampling of scores. Defaults to True, if False
            this function returns a collections.defaultdict[metric: list of values for each observation for each subscore]``
        newline_sep:(default=True) whether to add newline between sentences. This is essential for calculation rougeL
        on multi sentence summaries (CNN/DM dataset).

    Returns:
         Dict[score: value] if aggregate else defaultdict(list) keyed by rouge_keys

    """
    scorer = rouge_scorer.RougeScorer(rouge_keys, use_stemmer=use_stemmer)
    aggregator = scoring.BootstrapAggregator()
    for pred, tgt in zip(tgt_lns, pred_lns):
        # rougeLsum expects "\n" separated sentences within a summary
        if newline_sep:
            pred = add_newline_to_end_of_each_sentence(pred)
            tgt = add_newline_to_end_of_each_sentence(tgt)
        scores = scorer.score(pred, tgt)
        aggregator.add_scores(scores)

    if bootstrap_aggregation:
        result = aggregator.aggregate()
        if return_precision_and_recall:
            return extract_rouge_mid_statistics(result)  # here we return dict
        else:
            return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}

    else:
        return aggregator._scores  # here we return defaultdict(list)


def compute_metrics(pred):
    
    labels = pred.label_ids
    preds = pred.predictions
    #preds = pred.predictions.argmax(-1)
    #precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    #acc = accuracy_score(labels, preds)
    model_name='google/pegasus-gigaword'
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    def non_pad_len(tokens: np.ndarray) -> int:
        return np.count_nonzero(tokens != tokenizer.pad_token_id)

    def decode_pred(pred: EvalPrediction) -> Tuple[List[str], List[str]]:
        pred_str = tokenizer.batch_decode(pred.predictions, skip_special_tokens=True)
        label_str = tokenizer.batch_decode(pred.label_ids, skip_special_tokens=True)
        pred_str = lmap(str.strip, pred_str)
        label_str = lmap(str.strip, label_str)
        return pred_str, label_str

    def summarization_metrics(pred: EvalPrediction) -> Dict:
        pred_str, label_str = decode_pred(pred)
        rouge: Dict = calculate_rouge(pred_str, label_str)
        summ_len = np.round(np.mean(lmap(non_pad_len, pred.predictions)), 1)
        rouge.update({"gen_len": summ_len})
        return rouge

    compute_metrics_fn = summarization_metrics
    
    return {
    #        'accuracy': acc,
     #       'f1': f1,
      #      'precision': precision,
            'rouge' : compute_metrics_fn
       #     'recall': recall
            }

In [ ]:
def freeze(model):
  for param in model.parameters():
    param.requires_grad = False


# Loss

In [ ]:
def cross_entropy_loss(logits, labels,label_smoothing,pad_token_id):
    lprobs = F.log_softmax(logits, dim=-1)
    student_lm_loss, _ = label_smoothed_nll_loss(
                lprobs, labels, label_smoothing, ignore_index=pad_token_id
            )
    return student_lm_loss

def label_smoothed_nll_loss(lprobs, target, epsilon, ignore_index=-100):
    """From fairseq"""
    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(-1)
    nll_loss = -lprobs.gather(dim=-1, index=target)
    smooth_loss = -lprobs.sum(dim=-1, keepdim=True)
    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        nll_loss.masked_fill_(pad_mask, 0.0)
        smooth_loss.masked_fill_(pad_mask, 0.0)
    else:
        nll_loss = nll_loss.squeeze(-1)
        smooth_loss = smooth_loss.squeeze(-1)

    nll_loss = nll_loss.sum()  # mean()? Scared to break other math.
    smooth_loss = smooth_loss.sum()
    eps_i = epsilon / lprobs.size(-1)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss, nll_loss

def calc_hidden_loss(attention_mask, hidden_states, hidden_states_T, matches, normalize_hidden):
    """MSE(student_hid, teacher_hid[matches]). Called "Intermediate supervision" in paper. Inspired by TinyBERT."""
    msg = "expected list or tuple for hidden_states, got tensor of shape: "
    mask = attention_mask.to(hidden_states[0])
    valid_count = mask.sum() * hidden_states[0].size(-1)
    student_states = torch.stack([hidden_states[i] for i in range(len(matches))])
    teacher_states = torch.stack([hidden_states_T[j] for j in matches])
    if normalize_hidden:
        student_states = F.layer_norm(student_states, student_states.shape[1:])
        teacher_states = F.layer_norm(teacher_states, teacher_states.shape[1:])
    mse = F.mse_loss(student_states, teacher_states, reduction="none")
    masked_mse = (mse * mask.unsqueeze(0).unsqueeze(-1)).sum() / valid_count
    return masked_mse


def calc_ce_loss(mask, s_logits, t_logits,temperature=2):
    """Copy pasted from distillbert (transformers/examples/distillation/)"""
    # mask has False at padding_idx
    sel_mask = mask[:, :, None].expand_as(s_logits)

    #print(sel_mask.shape ,s_logits.shape ,t_logits.shape )
    
    vocab_size = s_logits.size(-1)
    s_logits_slct = torch.masked_select(s_logits, sel_mask)  # (bs * seq_length * voc_size) modulo the 1s in mask
    t_logits_slct = torch.masked_select(t_logits, sel_mask)  # (bs * seq_length * voc_size) modulo the 1s in mask
    s_logits_slct = s_logits_slct.view(-1, vocab_size)  # (bs * seq_length, voc_size) modulo the 1s in mask
    t_logits_slct = t_logits_slct.view(-1, vocab_size)  # (bs * seq_length, voc_size) modulo the 1s in mask
    ce_loss_fct = nn.KLDivLoss(reduction="batchmean")
    loss_ce = (
        ce_loss_fct(
            F.log_softmax(s_logits_slct / temperature, dim=-1),
            F.softmax(t_logits_slct / temperature, dim=-1),
        )
        * (temperature) ** 2 
    )
    return loss_ce

def shift_tokens_right(input_ids, pad_token_id):
    """Shift input ids one token to the right, and wrap the last non pad token (usually <eos>)."""
    prev_output_tokens = input_ids.clone()
    #print(pad_token_id, input_ids)
    x= (input_ids.ne(pad_token_id).sum(dim=1) - 1)
    index_of_eos = x.unsqueeze(-1)
    prev_output_tokens[:, 0] = input_ids.gather(1, index_of_eos).squeeze()
    prev_output_tokens[:, 1:] = input_ids[:, :-1]
    return prev_output_tokens

def blended_loss(teacher,student,batch,labels,e_layer_ids, d_layer_ids,pad_token_id): #TODO Calculate the total loss
        
        alpha_hid = 0.2
        alpha_ce = 0.8
        alpha_mlm= 0.8
        
        student_encoder_layers = student.config.encoder_layers
        student_decoder_layers = student.config.decoder_layers
        teacher_encoder_layers = teacher.config.encoder_layers
        teacher_decoder_layers = teacher.config.decoder_layers

        e_matches = get_layers_to_supervise(
                    n_student=len(e_layer_ids), n_teacher=teacher_encoder_layers
                )
        d_matches = get_layers_to_supervise(
                    n_student=len(d_layer_ids), n_teacher=teacher_decoder_layers
                )
        
        
        different_base_models = not (student is None or teacher == student)
        do_calc_hidden_loss = (not different_base_models) and alpha_hid > 0
        different_encoder = different_base_models or (student.config.encoder_layers != teacher.config.encoder_layers)
        
        input_ids,src_mask = batch['input_ids'],batch['attention_mask']
        decoder_input_ids = shift_tokens_right(labels, pad_token_id)
        
        student_outputs = student(input_ids,
            attention_mask=src_mask,
            decoder_input_ids=decoder_input_ids,
            output_hidden_states=True,
            output_attentions=False,
            use_cache=False)
        
        lm_logits = student_outputs["logits"]
        label_smoothing = 0.1
        student_loss = cross_entropy_loss(lm_logits, labels,label_smoothing,pad_token_id)

        def zero_tensor():
            return torch.tensor(0.0).type_as(student_loss)
        
        teacher_enc_outputs = student_outputs[
            "encoder_last_hidden_state"
        ]  # use this unless self.different_base_models
        hid_loss_enc, hid_loss_dec = zero_tensor(), zero_tensor()
        
        if different_encoder:  # compute encoder hidden state loss
            all_teacher_encoder_outputs = teacher.get_encoder()(
                input_ids,
                attention_mask=src_mask,
                output_hidden_states=True,
            )
            #print(all_teacher_encoder_outputs['hidden_states'].shape)
            if different_base_models:
                teacher_enc_outputs = all_teacher_encoder_outputs["last_hidden_state"]
            elif do_calc_hidden_loss:
                hid_loss_enc = calc_hidden_loss(
                    src_mask,
                    student_outputs["encoder_hidden_states"],
                    all_teacher_encoder_outputs["hidden_states"],
                    e_matches,
                    normalize_hidden=True,
                )
        # decoder_input_ids for teacher [8,1] (zeros)
        #decodeIds = torch.cuda.LongTensor([0,0,0,0,0,0,0,0]).reshape(1,8)
        teacher_outputs = teacher(
            input_ids,
            attention_mask=src_mask,
            encoder_outputs=(teacher_enc_outputs,),
            decoder_input_ids= decoder_input_ids,
            output_hidden_states=do_calc_hidden_loss,
            use_cache=False,  # since we are not passing labels, never let this default to True
        )
        dec_mask = decoder_input_ids.ne(pad_token_id)
        loss_ce = calc_ce_loss(dec_mask, lm_logits, teacher_outputs["logits"])
        if do_calc_hidden_loss:  # Intermediate supervision of decoder hidden states
            hid_loss_dec = calc_hidden_loss(
                dec_mask,
                student_outputs["decoder_hidden_states"],
                teacher_outputs["decoder_hidden_states"],
                d_matches,
                normalize_hidden=True,
            )
        losses = {'ce':loss_ce, 'student loss KD':student_loss, 'Hidden': hid_loss_dec}
        blended_loss = (
            alpha_ce * loss_ce
            + alpha_mlm * student_loss
            + alpha_hid * (hid_loss_enc + hid_loss_dec)
        )
        return blended_loss, losses

        

#CallBacks


In [ ]:
import re

_camel_re1 = re.compile('(.)([A-Z][a-z]+)')
_camel_re2 = re.compile('([a-z0-9])([A-Z])')
def camel2snake(name):
    s1 = re.sub(_camel_re1, r'\1_\2', name)
    return re.sub(_camel_re2, r'\1_\2', s1).lower()


class Callback():
    _order = 0
    def set_runner(self, run): self.run = run
    def __getattr__(self, k): return getattr(self.run, k)
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')
    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

In [ ]:
class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs = 0
        self.run.n_iter = 0
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs+=1./self.iters
        self.run.n_iter+=1
    def begin_epoch(self):
        self.run.n_epochs = self.epoch
        self.student.train()
        self.run.in_train = True
    def begin_validate(self):
        self.student.eval()
        self.run.in_train = False
    

class CancelTrainException(Exception): pass
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass

In [ ]:
from collections import Iterable
from fastprogress import master_bar, progress_bar
from pathlib import Path

def listify(o):
    if o is None: return [] 
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

class Runner():
    def __init__(self, cbs=None, cb_funcs=None):
        cbs = listify(cbs)
        for cbf in listify(cb_funcs):
            cb = cbf()
            setattr(self, cb.name, cb)
            cbs.append(cb)
        self.stop, self.cbs = False, [TrainEvalCallback()] + cbs
    @property
    def opt(self): return self.learn.opt
    @property
    def student(self): return self.learn.student
    @property
    def teacher(self): return self.learn.teacher
    @property
    def loss_func(self): return self.learn.loss_func
    @property
    def train_data(self): return self.learn.train_data
    @property
    def valid_data(self): return self.learn.valid_data
    
    def one_batch(self, xb, yb,pad_token_id):
        try: 
            self.xb, self.yb = xb, yb
            self('begin_batch')
            decoder_input_ids = shift_tokens_right(yb, pad_token_id)
            self.pred = self.student(xb['input_ids'],
                                    attention_mask=xb['attention_mask'],
                                    decoder_input_ids=decoder_input_ids,
                                    output_hidden_states=True,
                                    output_attentions=False,
                                    use_cache=False)
            self('after_pred')
            self.loss, losses = self.loss_func(self.teacher,self.student,xb,yb,self.learn.e_layer,self.learn.d_layer,pad_token_id)

            self('after_loss')
            wandb.log({f"{'train' if self.in_train else 'valid'} loss": self.loss})
            if not self.in_train: return
            self.loss.backward()
            self('after_backward')
            self.opt.step()
            self('after_step')
            self.opt.zero_grad()
            gc.collect()
            torch.cuda.empty_cache()
            #print(torch.cuda.get_device_name(0))
            #print('Memory Usage:')
            #print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
            #print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
            
        except CancelBatchException: self('after_cancel_batch')
        finally: self('after_batch')
    
    def all_batches(self, dl, pad_token_ids):
        self.iters = len(dl)
        try:
              for train_batch in progress_bar(dl, leave=False): 
                y = train_batch['labels'].to(device)
                x = {
                      'input_ids':train_batch['input_ids'].to(device),
                      'attention_mask':train_batch['attention_mask'].to(device)
                    }
                self.one_batch(x, y, pad_token_ids)
        except CancelEpochException: self('after_cancel_epoch')

    def fit(self, epochs, learn, pad_token_ids):
        self.epochs, self.learn, self.loss = epochs, learn, torch.tensor(0.)
        try: 
            for cb in self.cbs: cb.set_runner(self)
            self('begin_fit')
            self.learn.student.to(device)
            self.learn.teacher.to(device)
            for epoch in range(epochs):
                self.epoch = epoch
                if not self('begin_epoch'): self.all_batches(self.train_data, pad_token_ids)
                with torch.no_grad():
                    if not self('begin_validate'): self.all_batches(self.valid_data, pad_token_ids)
                self('after_epoch')
                self.save_checkpoint(epoch, f'/content/drive/MyDrive/GP/student_checkpoint_epoch_{epoch}.pt')
        except CancelTrainException: self('after_cancel_train')
        finally:
            self('after_fit')
            self.learn = None

    def save_checkpoint(self,EPOCH, PATH):
      torch.save({
            'epoch': EPOCH,
            'model_state_dict': self.learn.student.state_dict(),
            'optimizer_state_dict': self.learn.opt.state_dict(),
            'loss': self.loss,
            }, PATH)

    def __call__(self, cb_name):
        res = False
        for cb in sorted(self.cbs, key=lambda x: x._order): res = cb(cb_name) and res
        return res

In [ ]:
class Learner():
    def __init__(self, student, teacher, opt, loss_func, train_data, valid_data, e_layer, d_layer):
        self.student, self.teacher,self.opt, self.loss_func, self.train_data, self.valid_data, self.e_layer, self.d_layer = student, teacher, opt, loss_func, train_data, valid_data, e_layer, d_layer

In [ ]:
class AvgStats():
    def __init__(self, metrics, in_train): self.metrics, self.in_train = listify(metrics), in_train
    def reset(self):
        self.tot_loss, self.count = 0., 0
        self.tot_mets = [0.]*len(self.metrics)
    @property
    def all_stats(self): return [self.tot_loss.item()] + self.tot_mets
    @property
    def avg_stats(self): return [o/self.count for o in self.all_stats]
    
    def __repr__(self):
        if not self.count: return ''
        return f"{'train' if self.in_train else 'valid'}: {self.avg_stats}"
    def accumulate(self, run):
        bn = run.xb['input_ids'].shape[0]
        self.tot_loss+=run.loss*bn
        self.count+=bn
        for i, m in enumerate(self.metrics):
            self.tot_mets[i]+=m(run.pred, run.yb)*bn

In [ ]:
class AvgStatsCallBack(Callback):
    def __init__(self, metrics):
        self.train_stats, self.valid_stats = AvgStats(metrics, True), AvgStats(metrics, False)
    def begin_epoch(self):
        self.train_stats.reset()
        self.valid_stats.reset()
    def after_loss(self):
        stats = self.train_stats if self.in_train else self.valid_stats
        with torch.no_grad(): stats.accumulate(self.run)
    def after_epoch(self):
        print(self.train_stats)
        print(self.valid_stats)
        


In [ ]:
def roc(out, y):
    pass


In [ ]:
class Recorder(Callback):
    def begin_fit(self):
        self.lrs = [[] for _ in self.opt.param_groups]
        self.losses = []
    def after_batch(self):
        if not self.in_train: return
        for pg, lr in zip(self.opt.param_groups, self.lrs): lr.append(pg['lr'])
        self.losses.append(self.loss.detach().cpu())
    
    def plot_lr(self, pgid=-1): plt.plot(self.lrs[pgid])
    def plot_loss(self, skip_last=0): plt.plot(self.losses[:len(self.losses)-skip_last])
    def plot(self, skip_last=0, pgid=-1):
        losses = [o.item() for o in self.losses]
        lrs = self.lrs[pgid]
        n = len(losses)-skip_last
        plt.xscale('log')
        plt.plot(lrs[:n], losses[:n])

In [ ]:
class ParamScheduler(Callback):
    _order = 1
    def __init__(self, pname, sched_funcs): self.pname, self.sched_funcs = pname, sched_funcs
    def begin_fit(self):
        if not isinstance(self.sched_funcs, (list, tuple)):
            self.sched_funcs = [self.sched_funcs]*len(self.opt.param_groups)
    def set_param(self):
        assert len(self.opt.param_groups)==len(self.sched_funcs)
        for pg, f in zip(self.opt.param_groups, self.sched_funcs):
            pg[self.pname] = f(self.n_epochs/self.epochs)
    def begin_batch(self):
        if self.in_train: self.set_param()

In [ ]:
class LR_Find(Callback):
    _order = 1
    def __init__(self, max_iter=100, min_lr=1e-6, max_lr=10):
        self.max_iter, self.min_lr, self.max_lr = max_iter, min_lr, max_lr
        self.best_loss = 1e9
    def begin_batch(self):
        if not self.in_train: return
        pos = self.n_iter/self.max_iter
        lr = self.min_lr*(self.max_lr/self.min_lr)**pos
        for pg in self.opt.param_groups: pg['lr'] = lr
    def after_step(self):
        if self.n_iter>=self.max_iter or self.loss>self.best_loss*10:
            raise CancelTrainException()
        if self.loss<self.best_loss: self.best_loss = self.loss


In [ ]:
run = Runner(cb_funcs=[LR_Find, Recorder])
torch.cuda.empty_cache()
gc.collect()
def annealer(f):
    def _inner(start, end): return partial(f, start, end)
    return _inner
@annealer
def sched_lin(start, end, pos): return start + pos*(end-start)
import math
@annealer
def sched_cos(start, end, pos): return start + (1+math.cos(math.pi*(1-pos)))*(end-start)/2

@annealer
def sched_no(start, end, pos): return start

@annealer
def sched_exp(start, end, pos): return start*(end/start)**pos

torch.Tensor.ndim = property(lambda x: len(x.shape))

def combine_scheds(pcts, scheds):
    assert sum(pcts)==1.
    pcts = torch.tensor([0] + listify(pcts))
    assert torch.all(pcts>=0)
    pcts = torch.cumsum(pcts, 0)
    def _inner(pos):
        idx = (pos>=pcts).nonzero().max()
        actual_pos = (pos-pcts[idx])/(pcts[idx+1]-pcts[idx])
        return scheds[idx](actual_pos)
    return _inner
def pg_dicts(pgs): return [{'params': o} for o in pgs]
def cos_1cycle_anneal(start, high, end):
    return [sched_cos(start, high), sched_cos(high, end)]

from functools import partial

phases = [0.2, 0.8]
scheds = combine_scheds(phases, [sched_cos(1e-4, 5e-3), sched_cos(5e-3, 1e-3)])


In [ ]:
cbfs = [Recorder, partial(ParamScheduler, 'lr', scheds)]

In [ ]:
run = Runner(cb_funcs=cbfs)

In [ ]:
def get_learner(teacher, student, optimizer, loss_fcn ,train_dataloader , validation_dataloader,e_layers_list, d_layers_list,bsz=24):
     return Learner(student, teacher, optimizer, loss_fcn, train_dataloader , validation_dataloader,e_layers_list, d_layers_list) 

In [ ]:
def train_and_eval(epochs, cbfs):
    torch.cuda.empty_cache()
    # Teacher model and Tokenizer

    tokenizer = AutoTokenizer.from_pretrained('google/pegasus-gigaword')

    teacher = AutoModelForSeq2SeqLM.from_pretrained('google/pegasus-gigaword')
    
    student, e_layers_list, d_layers_list = create_student_with_configuration(
                                      teacher,
                                      e=4,
                                      d=4,
                                      copy_first_teacher_layers = False,
                                      save_path='./student')
    optimizer = AdamW(student.parameters(), lr=5e-5)

    learn = get_learner(teacher,student,optimizer,blended_loss,train_dataloader, validation_dataloader,e_layers_list, d_layers_list)

    gc.collect()
    torch.cuda.empty_cache()
    
    run = Runner(cb_funcs=cbfs)
    learn.student.train()
    freeze(learn.teacher)
    learn.teacher.eval()
    pad_token_id = tokenizer.pad_token_id
    
    ## ''' Train and Validate LOOP ''' ##
    run.fit(epochs, learn, pad_token_id)

    ## ''' Evaluation LOOP ''' ##
    with torch.no_grad():
      learn.student.eval()
      all_labels = []
      all_preds = []
      test_loss=[]
      for test_batch in test_dataloader:
        y = test_batch['labels'].to(device)
        x = {
              'input_ids':test_batch['input_ids'].to(device),
              'attention_mask':test_batch['attention_mask'].to(device)
            } 
                  
        prediction = learn.student.generate(**x)
              
        all_labels.append(y)
        all_preds.append(prediction)
        blended , losses = blended_loss(teacher,student,x,y,e_layers_list, d_layers_list,pad_token_id)
        test_loss.append(blended.item())
      del learn
      gc.collect()
      test_losses = torch.mean(torch.tensor(test_loss))
      print('test_loss: ', test_losses.item())
      preds = [tokenizer.decode(pred[0]) for pred in all_preds]
      lbls = [tokenizer.decode(lbl[0]) for lbl in all_labels]
      rouge_score = calculate_rouge(pred_lns=preds,tgt_lns=lbls)
      return rouge_score


In [ ]:
def sigmoid(x): return 1/(1+np.exp(-x))
preds = train_and_eval(4,cbfs)
wandb.log(preds)
preds